In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import json
import re
import shap
import lightgbm as lgb
import itertools
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

from utils.utils import *
from utils.pre_processing import *
from utils.nn import *
from utils.nn_utils import *

In [2]:
df = load_data('data/df_tags','data/final_reviews')
X_train, X_test, X_val, y_val, y_train, y_test = init_df(df,test_size=0.2,target_col='target',stratify=True,scaling=True,oversampling=True,undersampling=False)
X_train, X_test, X_val, to_remove = remove_highly_correlated_features(X_train, X_test, X_val, threshold=0.9)
df = df.drop(columns=df.columns[to_remove])

In [3]:
feature_names = df.columns[:-1]  # Assuming the last column is the target

# Create LightGBM datasets
lgb_train = lgb.Dataset(X_train, label=y_train, feature_name=list(feature_names))
lgb_val = lgb.Dataset(X_val, label=y_val, feature_name=list(feature_names), reference=lgb_train)

# Define the model parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 1
}

def f1_eval(y_pred, dataset):
    y_true = dataset.get_label()
    y_pred_binary = np.round(y_pred)
    return 'f1', f1_score(y_true, y_pred_binary), True

# Train the model
model = lgb.train(
    params, 
    lgb_train, 
    num_boost_round=100, 
    valid_sets=[lgb_train, lgb_val], 
    valid_names=['train', 'val'],
    feval=f1_eval
)

# Make predictions on the validation set
y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)
y_val_pred_binary = np.round(y_val_pred)

# Calculate evaluation metrics
accuracy = accuracy_score(y_val, y_val_pred_binary)
precision = precision_score(y_val, y_val_pred_binary)
recall = recall_score(y_val, y_val_pred_binary)
f1 = f1_score(y_val, y_val_pred_binary)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Get feature importance
importance = model.feature_importance()
feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importance})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Return the feature importance and metrics
evaluation_metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1
}

quantile_to_keep = 0.5
min_imp = feature_importance['Importance'].quantile(quantile_to_keep)
imp_feats = feature_importance.loc[feature_importance['Importance'] >= min_imp]['Feature']

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2705, number of negative: 2705
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006850 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75579
[LightGBM] [Info] Number of data points in the train set: 5410, number of used features: 304
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.8263305322128851
Precision: 0.1
Recall: 0.17307692307692307
F1 Score: 0.1267605633802817


In [4]:
feature_importance.sort_values('Importance')['Feature'].head(20)

254                        Werewolves
290                      Outbreak Sim
289    Intentionally Awkward Controls
56                     Hack and Slash
287                       Programming
286                     Warhammer 40K
57                             Nudity
278                             Party
293                         Benchmark
283                       Time Attack
282                       Agriculture
280                 Time Manipulation
63                   Great Soundtrack
294                     Political Sim
82                       Early Access
205                        Minimalist
258                      Auto Battler
259          Asynchronous Multiplayer
35                             Aliens
285                             Naval
Name: Feature, dtype: object

In [5]:
df = load_data('data/df_tags','data/final_reviews')

X_train, X_test, y_train, y_test = init_df(df,test_size=0.2,target_col='target',stratify=True,scaling=False,oversampling=False,undersampling=False,val=False)
X_train = X_train.values
X_test = X_test.values

X_train_reduced = np.delete(X_train, to_remove, axis=1)
X_test_reduced = np.delete(X_test, to_remove, axis=1)

reduced_X_train = X_train[:, imp_feats.index]
reduced_X_test = X_test[:, imp_feats.index]

In [6]:
#constantes

input_size = reduced_X_train.shape[1]
output_size = 1
hidden_activation = 'relu'
output_activation='sigmoid'
criterion = 'BCEWithLogitsLoss'
device = 'cpu'
pos_weight = torch.tensor((1 - df['target']).sum()/df['target'].sum())

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import pandas as pd

K = 5

# Define your hyperparameters
hidden_layers = [1, 2]
hidden_layer_max_sizes = [256, 128, 64, 32, 16]
lrs = [0.01, 0.001, 0.0001, 0.00001]
patience = [10]
num_epochs = 1000

if K > 1:
    k_fold = KFold(n_splits=K)  # Specify K as the number of folds

results = []

for h_layers, h_size, lr, pat in itertools.product(hidden_layers, hidden_layer_max_sizes, lrs, patience):

    hidden_layers_struct = get_layers(h_layers,h_size)
    
    fold_metrics = {'Accuracy': [], 'Precision': [], 'Recall': [], 'F1': []}

    if K == 1:
        dim = int(reduced_X_train.shape[0])
        dim_thres = int(dim*0.5)
        fold_generator = [(np.arange(0,dim_thres),np.arange(dim_thres,dim))]
    else:
        fold_generator = k_fold.split(reduced_X_train)
        
    for train_index, val_index in fold_generator:
        # Split data into train and validation set for this fold
        X_train_fold, X_val_fold = reduced_X_train[train_index], reduced_X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        scaler = StandardScaler()
        X_train_fold = scaler.fit_transform(X_train_fold)
        X_val_fold = scaler.transform(X_val_fold)
        
        # smote = SMOTE(random_state=42)
        # X_train_fold, y_train_fold = smote.fit_resample(X_train_fold, y_train_fold)

        # undersampler = RandomUnderSampler(random_state=42)
        # X_train_fold, y_train_fold = undersampler.fit_resample(X_train_fold, y_train_fold)

        # Create datasets and model
        train_loader, _, val_dataset = create_datasets(X_train_fold, y_train_fold, X_test, y_test, X_val_fold, y_val_fold, val=True, batch_size=320)
        model = NeuralNetwork(input_size, hidden_layers_struct, output_size, pos_weight, lr, device=device, hidden_activation=hidden_activation, output_activation=output_activation, criterion=criterion)

        # Train the model
        train_nn(model, train_loader, val_dataset, num_epochs=num_epochs, skip=50, patience=pat)

        # Evaluate the model on validation set for this fold
        y_pred = predict_nn(model, val_dataset)
        accuracy, precision, recall, f1 = eval_predict_nn(y_val_fold, y_pred)

        # Store metrics for this fold
        fold_metrics['Accuracy'].append(accuracy)
        fold_metrics['Precision'].append(precision)
        fold_metrics['Recall'].append(recall)
        fold_metrics['F1'].append(f1)

    # Calculate mean metrics across all folds
    mean_accuracy = np.mean(fold_metrics['Accuracy'])
    mean_precision = np.mean(fold_metrics['Precision'])
    mean_recall = np.mean(fold_metrics['Recall'])
    mean_f1 = np.mean(fold_metrics['F1'])

    if pos_weight is None:
        pos_weights_dict = 0
    else:
        pos_weights_dict = 1

    # Store hyperparameters and mean metrics in results list
    results.append({
        'Hidden Layers': h_layers,
        'Hidden Layer Size': h_size,
        'Learning Rate': lr,
        'Mean Accuracy': mean_accuracy,
        'Mean Precision': mean_precision,
        'Mean Recall': mean_recall,
        'Mean F1': mean_f1
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('new_model_results_pos_weight_CV.csv')

  0%|          | 0/1000 [00:00<?, ?it/s]

Early stopping at epoch 11
Accuracy: 0.9145658263305322
Precision: 0.10344827586206896
Recall: 0.07894736842105263
F1: 0.08955223880597014


  0%|          | 0/1000 [00:00<?, ?it/s]

Early stopping at epoch 12
Accuracy: 0.8949579831932774
Precision: 0.08823529411764706
Recall: 0.06382978723404255
F1: 0.07407407407407407


  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 050: | Train Loss: 1.372 | Val Loss: 1.3923 | Train F1: 0.1025| Val F1: 0.1098
Epoch 100: | Train Loss: 1.370 | Val Loss: 1.3916 | Train F1: 0.1052| Val F1: 0.1136
Epoch 150: | Train Loss: 1.370 | Val Loss: 1.3908 | Train F1: 0.0977| Val F1: 0.1055
Epoch 200: | Train Loss: 1.368 | Val Loss: 1.3899 | Train F1: 0.1162| Val F1: 0.1043
Epoch 250: | Train Loss: 1.368 | Val Loss: 1.3889 | Train F1: 0.1035| Val F1: 0.0752
Epoch 300: | Train Loss: 1.366 | Val Loss: 1.3880 | Train F1: 0.1046| Val F1: 0.0952
Epoch 350: | Train Loss: 1.365 | Val Loss: 1.3868 | Train F1: 0.1049| Val F1: 0.1270
Epoch 400: | Train Loss: 1.362 | Val Loss: 1.3855 | Train F1: 0.1239| Val F1: 0.1404
Epoch 450: | Train Loss: 1.360 | Val Loss: 1.3842 | Train F1: 0.1217| Val F1: 0.0417
Epoch 500: | Train Loss: 1.361 | Val Loss: 1.3826 | Train F1: 0.0684| Val F1: 0.0435
Epoch 550: | Train Loss: 1.356 | Val Loss: 1.3810 | Train F1: 0.0892| Val F1: 0.0444
Epoch 600: | Train Loss: 1.356 | Val Loss: 1.3792 | Train F1: 0.1

C:\Users\eduar\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\eduar\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\eduar\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 050: | Train Loss: 1.361 | Val Loss: 1.4409 | Train F1: 0.1139| Val F1: 0.1309
Epoch 100: | Train Loss: 1.359 | Val Loss: 1.4400 | Train F1: 0.1038| Val F1: 0.1297
Epoch 150: | Train Loss: 1.359 | Val Loss: 1.4389 | Train F1: 0.1071| Val F1: 0.1357
Epoch 200: | Train Loss: 1.352 | Val Loss: 1.4378 | Train F1: 0.1188| Val F1: 0.1408
Epoch 250: | Train Loss: 1.353 | Val Loss: 1.4366 | Train F1: 0.1101| Val F1: 0.1485
Epoch 300: | Train Loss: 1.350 | Val Loss: 1.4354 | Train F1: 0.1271| Val F1: 0.1768
Epoch 350: | Train Loss: 1.344 | Val Loss: 1.4341 | Train F1: 0.1223| Val F1: 0.1688
Epoch 400: | Train Loss: 1.344 | Val Loss: 1.4327 | Train F1: 0.1311| Val F1: 0.1739
Epoch 450: | Train Loss: 1.342 | Val Loss: 1.4313 | Train F1: 0.1285| Val F1: 0.1600
Epoch 500: | Train Loss: 1.338 | Val Loss: 1.4297 | Train F1: 0.1353| Val F1: 0.1681
Epoch 550: | Train Loss: 1.337 | Val Loss: 1.4283 | Train F1: 0.1197| Val F1: 0.1724
Epoch 600: | Train Loss: 1.337 | Val Loss: 1.4267 | Train F1: 0.1

In [13]:
results_df.sort_values('Mean F1',ascending=False)

,Hidden Layers,Hidden Layer Size,Learning Rate,Mean Accuracy,Mean Precision,Mean Recall,Mean F1
15,1,32,0.00001,0.877838,0.143366,0.210196,0.156218
14,1,32,0.00010,0.884007,0.123480,0.157319,0.132234
17,1,16,0.00100,0.872241,0.113380,0.169725,0.130988
6,1,128,0.00010,0.903898,0.141898,0.132648,0.130911
13,1,32,0.00100,0.891851,0.124687,0.140156,0.125699
38,2,16,0.00010,0.868883,0.115275,0.172628,0.123349
2,1,256,0.00010,0.901376,0.122138,0.130779,0.120866
11,1,64,0.00001,0.881207,0.107680,0.163235,0.120601
10,1,64,0.00010,0.893812,0.112114,0.136042,0.117496
26,2,128,0.00010,0.916786,0.145564,0.103722,0.115545
